In [ ]:
from IPython.display import clear_output # clear output
from PIL import Image # image processing
import numpy as np # linear algebra
from tqdm import tqdm # progress bar
import os # file handling
import sqlite3 # database

In [12]:
# Connexion à la base de données SQLite (création si elle n'existe pas)
conn = sqlite3.connect('../Dataset/dataset.db')
cursor = conn.cursor()

In [ ]:
# Suppression de la table photo si elle existe
# Si besoin de reset la table photo
cursor.execute("DROP TABLE IF EXISTS Photo;")

In [ ]:
# Création de la table avec les infos des photos
cursor.execute('''
CREATE TABLE IF NOT EXISTS Photo (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Label VARCHAR(1),
    Repartition VARCHAR(10),
    Chemin VARCHAR(100),
    Tableau_Pixels TEXT
)
''')

In [ ]:
# Transformation des photos en dataframe
def photo_to_dataframe(chemins_photos: str, label: str, separation: str, type_data: str):
    """Transforme les photos en tableau de pixels et les sauvegarde dans un fichier csv

    Args:
        chemins_photos (str): Chemin des photos (on veut le dossier contenant les photos)
        label (str): label de la photo (ce qui va être prédit)
        separation (str): séparation des données (train, test, validation)
    """
    
    # Affichage des dossiers en cours de traitement 
    print("..\Dataset")
    print("    \\" + type_data)
    print("      \\" + separation)
    print("        \\" + label)
    
    # Parcours et transformation des photos avec barre de progression 
    with tqdm(total=len(chemins_photos), desc="Proccessing" , position=0, leave=True) as pbar:
        for chemin in chemins_photos:
            
            pbar.set_postfix(img=chemin[-10:]) # Mettre à jour la barre de progression avec des informations supplémentaires
            
            img = Image.open(chemin) # 300 x 300
            nouvelle_taille = (50, 50) # 2500 
            img_redimensionnee = img.resize(nouvelle_taille) # Redimensionner l'image
            nb_image = img_redimensionnee.convert('L') # Convertion en noir et blanc
            
            #  Récupération de la liste des pixels
            tab = []
            for i in range(nb_image.size[1]):
                row = []
                for y in range(nb_image.size[0]):
                    # Récupérer la couleur du pixel
                    row.append(nb_image.getpixel((y, i)))
                # tab.append(np.array(row))
                tab.append(row)
                
            tab_numpy = np.array(tab) / 255 # mise à l'échelle des données
            tab_numpy_flatten = tab_numpy.flatten() # Transformation en tableau 1D
            tab_numpy_flatten_str = np.array2string(tab_numpy_flatten, separator=",", threshold=np.inf)
            
            # insertion des données dans la base de données
            cursor.execute('INSERT INTO Photo (Label, Repartition, Chemin, Tableau_Pixels) VALUES (?, ?, ?, ?)', (label, separation, chemin.replace('\\', '/'), tab_numpy_flatten_str.replace('\n', '')))
                
            pbar.update(1) # Mettre à jour la barre de progression
            
        # Validation de l'insertion des données 
        conn.commit()
    
    clear_output()  # Effacer l'output précédent (sinon pb d'affichage de la barre de progression)

In [16]:
# Parcours des dossiers et sous-dossiers pour recréer le dataset (recursion)
def recreate_dataset(chemin: str, profondeur: int, label: str, separation: str, type_data: str):
    """Réecriture du dataset: transformation des photos en tableau de pixels et les sauvegarde dans un fichier csv
    Cette fonction est récursive et doit être executer sur un dossier contenant la même arborescence que le dossier ../Dataset

    Args:
        chemin (str): chemin du dataset
        profondeur (int): profondeur de la recursion (elle permet aussi de récupérer les labels et les séparations)
        label (str): label de la photo (ce qui va être prédit)
        separation (str): séparation des données (train, test, validation)
    """
    dossiers = [f for f in os.listdir(chemin) if os.path.isdir(os.path.join(chemin, f))]
    if len(dossiers) == 0: # Si aucun dossier n'est trouvé, on est dans le cas de base
        fichiers_png = [os.path.join(chemin, f) for f in os.listdir(chemin) if f.endswith('.png')] # on recupère et stocke les fichiers .png
        photo_to_dataframe(fichiers_png, label, separation, type_data) # On transforme les photos en tableau en df qu'on save dans un fichier.csv
    else : # Sinon, on continue de parcourir les dossiers
        for dossier in dossiers:
            if type_data == 0:
                type_data = dossier
            if profondeur == 1:
                separation = dossier
            if profondeur == 2:
                label = dossier
            recreate_dataset(os.path.join(chemin, dossier), profondeur + 1, label, separation, type_data)

In [ ]:
# Appel de la fonction de parcours des dossiers
recreate_dataset("..\Dataset", 0, "truc", "bidule", "dataset")
# La valeur "dataset" comme 5ème argument est obligatoire pour ne pas avoir de fichier en plus ...

..\Dataset
    \dataset
      \test
        \B


Proccessing:   6%|▌         | 53/877 [00:00<00:07, 104.72it/s, img=000053.png]


KeyboardInterrupt: 

In [18]:
conn.close() # Fermeture de la connexion à la base de données SQLite